In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns

In [2]:
data = open("WhatsApp_chats.txt", "r")

In [3]:
headers = ["Month", "Day", "Year", "Hour", "Minute", "Author", "Message"]
messages = []
i = 0 
for message in data:  # This is so inefficient
    if "<Media omitted>" in message or i == 0: 
        i += 1
        continue
    try:
        entry = []

        index = message.find("/")
        entry.append(int(message[:message.find("/")])) 
        message = message[message.find("/") + 1:]
        entry.append(int(message[:message.find("/")]))
        message = message[message.find("/") + 1:]
        entry.append(int(message[:message.find(",")]))
        message = message[message.find(",") + 1:]
        entry.append(int(message[:message.find(":")].strip()))
        message = message[message.find(":") + 1:]
        entry.append(int(message[:message.find(" ")]))
        message = message[message.find(" ") + 1:]

        if message.startswith("PM"):
            entry[-2] += 12
            

        message = message[message.find(" - ") + 3:]
        entry.append(message[:message.find(":")])
        entry.append(message[message.find(":") + 1:].strip())
        messages.append(entry)
    except:
        messages[-1][-1] += "\n" + message
    else:
        continue

In [4]:
df = pd.DataFrame(messages, columns=headers)
print(df)

       Month  Day  Year  Hour  Minute    Author  \
0          8    6    19     2       2     Caleb   
1          8   18    19    15      37     Caleb   
2          8   18    19    15      43  Marianna   
3          8   18    19    15      43  Marianna   
4          8   18    19    15      44  Marianna   
...      ...  ...   ...   ...     ...       ...   
42417     12    9    19    23      55  Marianna   
42418     12    9    19    23      55  Marianna   
42419     12    9    19    23      55     Caleb   
42420     12    9    19    23      55     Caleb   
42421     12    9    19    23      56     Caleb   

                                                 Message  
0                                             Hello! 👉🙃👉  
1                            Let me know if you get this  
2                                              I got it!  
3      Idk what’s up with my phone it turns your +45 ...  
4                        I’m gonna try just typing it in  
...                              

In [5]:
full_texts = df.groupby("Author").agg(lambda x: '\n'.join(x))

In [6]:
text = full_texts["Message"].to_list()[1]

In [7]:
miss_yous = df[df["Message"].str.contains("iss you")]
miss_you_counts = miss_yous.groupby("Hour").size().to_dict()
print(miss_you_counts)
hours = [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,1,2,3,4,5,6,7]
counts = []
for hour in hours:
    try:
        counts.append(miss_you_counts[hour])
    except:
        counts.append(0)

bar = plt.bar([str(hour) + ' PM' if hour > 12 else str(hour) + " AM" for hour in hours], counts)


{1: 4, 2: 3, 5: 1, 6: 1, 7: 1, 12: 6, 13: 2, 14: 1, 15: 3, 16: 6, 18: 4, 19: 4, 20: 4, 21: 6, 22: 7, 23: 11, 24: 1}


NameError: name 'plt' is not defined

In [7]:
%matplotlib inline

import numpy as np
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop

Using Theano backend.


In [ ]:
# Q1: What is the purpose of this block? When is `char_indices` used? What about `indices_char`?
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Q2: What is the purpose of this block? What does the `seqlen` and `step` parameters do?
seqlen = 40
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

# Q3: What about this block? What is `x` and what is `y`? Why do they have this dimensionality?
x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    # Q3a: What happens in this loop?
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


# Q4: Here we build the model. What does the `return_sequences` argument do? Why the dense layer at the end?
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))
model.add(Dense(len(chars), activation='softmax'))


model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

print("Compiled!")
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(1000):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback])

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '2813' (I am process '3848')


Compiled!
Epoch 1/50
20460/20460 [==============================] - 36s 2ms/step - loss: 3.6015 - categorical_crossentropy: 3.6015 - accuracy: 0.1644

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y
You can have dreams
Yes!
How’s your ni"
y
You can have dreams
Yes!
How’s your ni    a              e                                                                       e  o  e             s            e  o           o     o  e                  e        e                   o                   i       ee   ee    e                                                          o       o     o        t                o      t                      ei                t  e                e           e         i                                     oe      n e      e      e                       e           n                                                         et o           t       t                                                                  e  te    e

Well I already have a tripped to the the and the wanting the store somether we best to do that to do the and the walse the beart the liner than the stayant and to do that in the mo the mean the stayant the worst but I got the best the stean that the stades than the little beart the steant and the liner the stades the store and the liner and the want the stades and to go that and it want the and the littare best and the and the and the best and the asterest and to do the and and the stean to the and the and the stadent the store we lot on toming as the beart and I stean the beart the stust and the and the beart the stind the weese the and to the wants to come and the tome worg the store somether on tome soon and the stadent somether well and I stades and to go that than the stind the peise the beargon so I there the and to the panerants on the peeses the worny to the anythere and the bean and we base and I the and the littare and the hand it and the wants that wound shoulg so the lond t

And I should get to got to see to student and hy something on the good all didn’t be something song
It’s pretty conered
It’s connesting to things to do this and it before on my study on the because we see is in the one that is expention that beport and it wald we got thoum
I’m not kay be
Have you book and the bead
You’re also wald be like the tood and certies times to go the sound and feels and the all though
It’s an alling so and be and the beading and it to have at to do or
It’s one of the now
I don’t don’t like you in the one the phone where the and the month so with me and it’s and made not can deminitent want to want to go to mander
Than work but I don’t been any and the like and wanting to the tomorrherious and an amans thind we were like we were good so I like something
What got to lest to have to have sounds so something than you something
Okay
Okay in it to frab to line out me that sounds things so that one on the because that soust and all thy bed
I this I guest and have to 


It was profonray omi with itgrnes
Mt in the bana saight to do that I’m fine this actually I was rooming better I’d torran Beha
Okay senceeh azes thow ir aslees midty foreet
As or listenere thel thingh we got co kind on yip
You just ose the mornon*
Ha
So teit to at proicless
Yes I CIBLAKS TVIP
yon’t exart you is toied way co this ry rine some lable a up you momed to do good barest is onl’t ham torream
: so Cean not so know
Sucry/a timen
I get worh I guyfjist on the asr
Hhat those are good rusomend”
Oh is homework
She on that?
Upkah with that timeuter
I’m doatry is boughnd wey
And she sater beerdroom?
(And the last time
I have it though I dony is mode to wa one ammare yof if there add malling *:b garding and see is good
I’m goenie you got solning sousking we and lrat like movie!  Mow
Sithed 30.
But I thoukumord
I that 3ierning mithed
Good exscuredny dexttle somes where antter alkouce we kis enalon
Emmere anything so I have it ofnesting mere
I may stap
Yes-booked hy bea brnoriths fart out

...

Okay I’m gonna stay and the best or an hour and then I should have to be bad
I have to be too hard to cheer them and I was a comp bio
I kind of think out in the has to be the day
But because I don’t think we were really home on the park and the best and have it
But I think that doesn’t exact to go to the sumber in the same people was a comp bio
I won’t say in his bailtiss the bad
I can do that to the semester woah that would be too think it’s because I don’t want to make me but I’d be semester remember for the past chicken else with us in my phone and I’m like at least to be semesting a park and then I have to go to me in And the one I’m sorry
I was blronp abroad my friends were part of my room on the compution when I was excited to had the first time and have to go out to go to dinner but I can wait to probably have to be the park
And no
No I think I’m not just a good bad when I have to do that more and not thing in chlam
So I’m hoping to a prof for so nice
I have to go back to you wo

 entire why davidson about Maggie last yea when I abroad the time
Stuff and angonts on the schata
He school like no
I realized some lors
Yeah after sleep if the “twk!
But woam just torgirup or time finalic like pleased now I’d like it
Kidding and I things only and still had any of the ovy
Would I dwoo?
Mostly you’re you another
We
I need to be dood
I’m too long knowive cerselcomalle
It has from the personal to think there trautab up in stuff so thisd yeah I’m gonna do anything for you/bart
Hey
And I think they weve
Inneed it agrymor overss for yoga!
There are 2 isn’t
It’s a davidson
It’s frights!?
Ge
Casers best way the way lunch
Miss you
so wow
That hand
I want to card up on frozmp of why
You’re at resebcking
You’re well if with eally dusre soint...
Oh no I’m going to dip I but I really have to bother
It’s kierday peop leavinka isson but he was my first breatherh the paper because we susse
Also
Sarah and, off it homework like that in years can shaon mostewherese in the metherg us to i

I don’t know what I was like what I didn’t want to see the class but I don’t want to see the day because I see you the states and the music and the particularly to the pass and I was allowed to be a song and was a little bit of the commute them and the probably still the probably and I was a little face and the students to me to be like ah that’s a little bit and hear and then the pass and the story and the cool that I didn’t have to be a lot of the states with them
I don’t think I’m gonna stay and the story
I don’t know what I don’t think I’ll be able to finish and still have a prof ask that the particularly
And the pa
----- diversity: 0.5
----- Generating with seed: "h my gosh
So many pups
Yeah no sorry I c"
h my gosh
So many pups
Yeah no sorry I can’t get somewhere to like them today so I might be much about them to see you and her so nice?
Here are you all about the day
Oh how we don’t want to go at the really started the forming ones the last time and not done that is good with th

And we can needed
Okay!
It’s funry trynge.sinmed her like this sonk
But I’m not going the time
Well for anything communidy name boilsiony all of the planting gader minman blanket into class last week no eally got me
You’re so newjad says walking though
Although it might have to be friend pale of not trying so NESINNTUL heggelonses
Grrakeap here we know if it’s class
I ended cope of music at Sarah is a regist
And is
NO GUH TTRISS MAAN UP Lina had
You’ll get and
I haven’t seen my lot nap scaphard call an tridying of a spratchs the last 20td certient yet how hirss
I’m not gonna ansoriousd and one puon because they are in next tigli bick skiwb things up with my technnained we’ve seen them
I had been questions ofill the slugchs
Evoryo
Class of pod vib
So am I like her uh I want to
I want paying if the num/stusclos house of ree in us them and enjoyed because I have that
Minicassuns
Interesting and day

Epoch 35/50
20460/20460 [==============================] - 43s 2ms/step - loss: 1.3360 - c

And I don’t want to go to the street in the story and the stuff to be a good time and I was also been happy to be able to the street the stuff to be able to see if I’m gonna be a song with my bed and I want to go to the street to the stuff to be able to have to start doing the considering your bed and I was like woah that’s a different conference is a little good to the same things
I want to go to the same time and the same things
I think I’m gonna get to stop the states
I don’t want to see them and I want to go to the gonna be a concert and I was like a doing and the most too
Yeah I’m gonna stay and it was a little be and the past in t
----- diversity: 0.5
----- Generating with seed: " about it she’s great
How has your early"
 about it she’s great
How has your early
They are satently a pretty things we can see how the end of the group is the conference the same time of the over and have a tota’t sent theu were away to be able to be a sent them
I wanted to be fires to phese is only and

In [ ]:
start_index = random.randint(0, len(text) - seqlen - 1)

# Q5: What does diversity do?
for diversity in [0.2, 0.5, 1.0]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + seqlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        # What is the dimensionality of `preds`? Why do we input `preds[0, -1]` to the `sample` function?
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
